<a href="https://colab.research.google.com/github/gagan3012/DetectingBias/blob/master/DetectingBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Feb 26 18:42:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!mkdir ~/.kaggle
!echo '{"username":"gagan2000","key":"73e5d9db85677b98fe016b25873727ec"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
!pip install -U -q kaggle

In [3]:
!kaggle datasets download -d snapcrack/all-the-news -p /content

 98% 239M/244M [00:02<00:00, 81.3MB/s]
100% 244M/244M [00:03<00:00, 84.3MB/s]


In [4]:
!unzip /content/all-the-news.zip -d /content/

Archive:  /content/all-the-news.zip
  inflating: /content/articles1.csv  
  inflating: /content/articles2.csv  
  inflating: /content/articles3.csv  


In [5]:
!pip install tensorflow==1.15 #CPU
!pip install tensorflow-gpu==1.15  # GPU
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 53.4MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 3.8MB 44.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=6c3269ec8f81364ca532ccb5ebd5dbe91c238118a186b78bb96c651f3505af27
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [6]:
gpt2.download_gpt2(model_name="124M")

gpt2.mount_gdrive()

Fetching checkpoint: 1.05Mit [00:00, 424Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.24Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 398Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:14, 33.8Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 302Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.25Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 2.78Mit/s]


Mounted at /content/drive


In [7]:
d1 = pd.read_csv('articles1.csv', encoding = 'utf-8')
d2 = pd.read_csv('articles2.csv', encoding = 'utf-8')
d3 = pd.read_csv('articles3.csv', encoding = 'utf-8')
data = pd.concat([d1, d2, d3])
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [9]:
data['publication'].value_counts()

Breitbart              23781
New York Post          17493
NPR                    11992
CNN                    11488
Washington Post        11114
Reuters                10710
Guardian                8681
New York Times          7803
Atlantic                7179
Business Insider        6757
National Review         6203
Talking Points Memo     5214
Vox                     4947
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64

In [10]:
data['content'] = data['content'].fillna("")
data['title'] = data['title'].fillna("")

In [11]:
final_right = data[data['publication'].str.contains("Fox News|New York Post|National Review|Breitbart")]
final_right = final_right[['title','content']]
final_right.shape

(51831, 2)

In [12]:
final_left = data[data['publication'].str.contains("Business Insider|Buzzfeed News|CNN|Guardian|New York Times|Talking Points|Memo|Vox|Washington Post")]
final_left = final_left[['title','content']]
final_left.shape

(60858, 2)

In [13]:
final_right['content'].to_csv("rightcontent.txt",header=None, index=None, sep='\t', mode='a')
final_left['content'].to_csv("leftcontent.txt",header=None, index=None, sep='\t', mode='a')

In [14]:
sess = gpt2.start_tf_sess()
file_name_1 = "rightcontent.txt"

In [ ]:
#for right content
gpt2.finetune(sess,
              dataset=file_name_1,
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name='run_right_1',
              print_every=100,
              sample_every=200,
              save_every=500,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


In [18]:
gpt2.generate(sess, run_name='run_right_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=False
              )

Allowing more immigrants to come into the United States will help create jobs, while allowing them to stay here in the hope that they will be able to find work while they are in the U. S.’s free youth labor market. ”                                                            
Allowing more immigrants to come into the United States will help us all.   But instead of helping the countless Americans who have been victims of crime, let me make clear: we will be victimized by one more criminal generation.                                                                 
Allowing more immigrants to come into the United States will help us all,” he said. ”We all have a responsibility to help them, but it will only benefit us if they assimilate into our country.” Trump also said he would not accept more Muslim immigrants. ”When I say, ’I am going to deport all Muslims,’ I mean, that’s not going to happen,” Trump said, as he addressed the crowd at the University of Phoenix. ”I want to be very
Al

In [20]:
gpt2.copy_checkpoint_to_gdrive(run_name='run_right_1')

In [ ]:
sess = gpt2.start_tf_sess()
file_name_2 = "leftcontent.txt"

In [19]:
#for left content
gpt2.finetune(sess,
              dataset=file_name_2,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run_left_1',
              print_every=10,
              sample_every=200,
              save_every=500,
              )

ValueError: ignored

In [ ]:
gpt2.generate(sess, run_name='run_left_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=True
              )